# Data Pre-processing

Since the image-text data extracted by the CLIP model has a large amount of feature data, it is stored as HDF5 files, which is convenient for subsequent data loading. The stored feature file contains the result of fusion of image features and text features.

In [ ]:
import numpy as np
import librosa
from note_seq.protobuf import music_pb2
from note_seq import sequences_lib
from note_seq import audio_io
from note_seq import midi_io
from pydub import AudioSegment
import tensorflow as tf
from note_seq import note_sequence_to_midi_file, NoteSequence, midi_to_note_sequence
from magenta.models.music_vae import TrainedModel, configs

import pandas as pd
import torch
import clip
from PIL import Image
import os
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
import note_seq

from tensorflow.keras.layers import Concatenate, Dense, LSTMCell, RNN, StackedRNNCells, Layer
from tensorflow.keras import Model
from sklearn.model_selection import train_test_split

from tensorflow.keras.optimizers import Adam
from tqdm import tqdm

import h5py
import json

In [ ]:
base_data = pd.read_csv("E:\\Project and Dissertation in Data Science\\dataset\\matched_data_TINYBERT.csv")
base_data

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201
...,...,...,...,...,...
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153


## CLIP Model

### Load Model

In [ ]:
# load pre-trained CLIP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

### Extract Image and Text Features

In [ ]:
image_base_path = "E:\\Project and Dissertation in Data Science\\dataset\\artemis_official_data\\art_images"

# 拼接图像文件名后缀
data['Artwork'] = data['Artwork'].apply(lambda x: x + '.jpg')

def preprocess_image(image_filename):
    image_path = os.path.join(image_base_path, image_filename)
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image).unsqueeze(0)
    return image

batch_size = 5  # 定义每次处理的批次大小
num_batches = len(data) // batch_size + (len(data) % batch_size != 0)

# 初始化存储特征和相似度的列表
all_image_features = []
all_text_features = []
all_similarities = []

for i in tqdm(range(num_batches), desc="Processing batches", total=num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(data))

    # 逐个处理批次中的图像和文本
    for idx in range(start_idx, end_idx):
        # 预处理图像
        image = preprocess_image(data['Artwork'].iloc[idx])

        # 处理对应的文本
        text = data['Art_Utterance'].iloc[idx]

        # 提取图像和文本特征
        with torch.no_grad():
            image_feature = torch.cat([model.encode_image(image.to(device))]).cpu().numpy()
            text_token = clip.tokenize([text], truncate=True).to(device)
            text_feature = model.encode_text(text_token).cpu().numpy()

        # 存储特征
        all_image_features.append(image_feature)
        all_text_features.append(text_feature)

        # 计算相似度
        similarity = cosine_similarity(image_feature, text_feature)[0][0]
        all_similarities.append(similarity)

# 合并所有批次的特征并存储到数据框中
data['image_features'] = all_image_features
data['text_features'] = all_text_features
data['similarities'] = all_similarities

# 打印结果和相似度
print(data.head())

Processing batches: 100%|██████████| 88733/88733 [2:56:44<00:00,  8.37it/s]  


                                             Artwork  \
0  vincent-van-gogh_portrait-of-madame-ginoux-l-a...   
1  vincent-van-gogh_portrait-of-madame-ginoux-l-a...   
2  vincent-van-gogh_portrait-of-madame-ginoux-l-a...   
3  vincent-van-gogh_portrait-of-madame-ginoux-l-a...   
4  vincent-van-gogh_portrait-of-madame-ginoux-l-a...   

                                       Art_Utterance           Music_Name  \
0  She seems very happy in the picture, and you w...    ABVYSaLu_VM_10-20   
1  This woman has really knotty hands which makes...  vnwKpQeza3A_320-330   
2  When looking at this woman, I am filled with c...    0VwX92X3iPc_30-40   
3  A woman looking at ease, peaceful, and satisfi...  kh6rmFg3U4k_480-490   
4  She looks like a lady from that past that migh...  -VI2IRq17rs_360-370   

                                       Music_Comment  Similarity_Score  \
0  Here we have a slow piano piece played in a ma...          0.791458   
1  This is a recording of two didgeridoos. They a...

In [ ]:
# 创建 HDF5 文件（'w' 代表写模式）
with h5py.File('data.h5', 'w') as hf:
    hf.create_dataset('image_features', data=np.array(data['image_features'].tolist()))
    hf.create_dataset('text_features', data=np.array(data['text_features'].tolist()))


In [ ]:
# 打开 HDF5 文件并读取数据
with h5py.File('data.h5', 'r') as hf:
    image_features = hf['image_features'][:]
    text_features = hf['text_features'][:]

# 将数组转换为列表（如果需要）并添加到 other_data DataFrame
base_data['image_features'] = list(image_features)
base_data['text_features'] = list(text_features)

In [ ]:
data2 = base_data
data2.tail()

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879,"[[0.4673, 0.1345, 0.04214, -0.2769, 0.1053, -0...","[[-0.04492, 0.182, 0.3486, -0.03268, -0.05658,..."
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782..."
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0...."
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ..."
443661,ivan-aivazovsky_sea-at-night-1861,The steep mountains and the moonlight provide ...,8q0An6WY7_c_10-20,The low quality recording features a theremin ...,0.761976,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.2, -0.1665, 0.2362, 0.0648, 0.2886, 0.014..."


## Feature Fusion Method

In [ ]:
# 检查每列的长度
image_feature_length = data2['image_features'].apply(len)
text_feature_length = data2['text_features'].apply(len)

# 检查是否每个样本的特征长度都相同
all_image_sizes_same = image_feature_length.nunique() == 1
all_text_sizes_same = text_feature_length.nunique() == 1

# 检查两列之间的长度是否相同
all_sizes_match = all(image_feature_length == text_feature_length)

print("所有图像特征的长度相同:", all_image_sizes_same)
print("所有文本特征的长度相同:", all_text_sizes_same)
print("两列特征的长度完全匹配:", all_sizes_match)

所有图像特征的长度相同: True
所有文本特征的长度相同: True
两列特征的长度完全匹配: True


In [ ]:
# 查看图像特征长度的分布
image_feature_length_distribution = data2['image_features'].apply(len).value_counts()
print("图像特征长度分布:\n", image_feature_length_distribution)

# 查看文本特征长度的分布
text_feature_length_distribution = data2['text_features'].apply(len).value_counts()
print("文本特征长度分布:\n", text_feature_length_distribution)


图像特征长度分布:
 1    443662
Name: image_features, dtype: int64
文本特征长度分布:
 1    443662
Name: text_features, dtype: int64


In [ ]:
print("Text features dimension:", text_features[0].shape[1])
print("Image features dimension:", image_features[0].shape[1])

Text features dimension: 512
Image features dimension: 512


In [ ]:
batch_size = 10  # 根据你的内存大小调整这个批大小
num_batches = (len(data2) + batch_size - 1) // batch_size

features_mean = []
features_weighted = []

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = min((i + 1) * batch_size, len(data2))

    # 计算批次内的向量平均
    batch_mean = (data2['image_features'][start_idx:end_idx] + data2['text_features'][start_idx:end_idx]) / 2
    features_mean.append(batch_mean)

    # 计算批次内的向量加权平均
    alpha = 0.6
    batch_weighted = alpha * data2['image_features'][start_idx:end_idx] + (1 - alpha) * data2['text_features'][start_idx:end_idx]
    features_weighted.append(batch_weighted)

# 将所有批次的结果合并回 DataFrame
data2['features_mean'] = np.concatenate(features_mean, axis=0)
data2['features_weighted'] = np.concatenate(features_weighted, axis=0)

In [ ]:
data2

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0...."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ..."
...,...,...,...,...,...,...,...,...,...
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879,"[[0.4673, 0.1345, 0.04214, -0.2769, 0.1053, -0...","[[-0.04492, 0.182, 0.3486, -0.03268, -0.05658,...","[[0.2112, 0.1582, 0.1954, -0.1548, 0.02435, 0....","[[0.2625, 0.1536, 0.1647, -0.1792, 0.04053, 0...."
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0...."
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0..."
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ...","[[0.0874, -0.1564, 0.09424, 0.1768, 0.284, 0.0...","[[0.10846, -0.1841, 0.0896, 0.22, 0.283, 0.012..."


In [ ]:
# 创建 HDF5 文件（'w' 代表写模式）
with h5py.File('data_mean_and_weighted.h5', 'w') as hf:
    hf.create_dataset('image_features', data=np.array(data2['image_features'].tolist()))
    hf.create_dataset('text_features', data=np.array(data2['text_features'].tolist()))
    hf.create_dataset('features_mean', data=np.array(data2['features_mean'].tolist()))
    hf.create_dataset('features_weighted', data=np.array(data2['features_weighted'].tolist()))


In [ ]:
# 打开 HDF5 文件并读取数据
with h5py.File('data_mean_and_weighted.h5', 'r') as hf:
    image_features = hf['image_features'][:]
    text_features = hf['text_features'][:]
    features_mean = hf['features_mean'][:]
    features_weighted = hf['features_weighted'][:]

# 将数组转换为列表（如果需要）并添加到 other_data DataFrame
base_data['image_features'] = list(image_features)
base_data['text_features'] = list(text_features)
base_data['features_mean'] = list(features_mean)
base_data['features_weighted'] = list(features_weighted)

In [ ]:
base_data

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0...."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ..."
...,...,...,...,...,...,...,...,...,...
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879,"[[0.4673, 0.1345, 0.04214, -0.2769, 0.1053, -0...","[[-0.04492, 0.182, 0.3486, -0.03268, -0.05658,...","[[0.2112, 0.1582, 0.1954, -0.1548, 0.02435, 0....","[[0.2625, 0.1536, 0.1647, -0.1792, 0.04053, 0...."
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0...."
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0..."
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ...","[[0.0874, -0.1564, 0.09424, 0.1768, 0.284, 0.0...","[[0.10846, -0.1841, 0.0896, 0.22, 0.283, 0.012..."


In [ ]:
# 查看图像特征长度的分布
image_feature_length_distribution = data2['image_features'].apply(len).value_counts()
print("图像特征长度分布:\n", image_feature_length_distribution)

# 查看文本特征长度的分布
text_feature_length_distribution = data2['text_features'].apply(len).value_counts()
print("文本特征长度分布:\n", text_feature_length_distribution)

# 查看图像特征长度的分布
image_text_feature_mean_length_distribution = data2['features_mean'].apply(len).value_counts()
print("图文均值特征长度分布:\n", image_text_feature_mean_length_distribution)

# 查看文本特征长度的分布
image_text_feature_weighted_length_distribution = data2['features_weighted'].apply(len).value_counts()
print("图文加权平均特征长度分布:\n", image_text_feature_weighted_length_distribution)


图像特征长度分布:
 1    443662
Name: image_features, dtype: int64
文本特征长度分布:
 1    443662
Name: text_features, dtype: int64
图文均值特征长度分布:
 1    443662
Name: features_mean, dtype: int64
图文加权平均特征长度分布:
 1    443662
Name: features_weighted, dtype: int64


In [ ]:
print("Text features dimension:", text_features[0].shape[1])
print("Image features dimension:", image_features[0].shape[1])
print("Image-Text features mean dimension:", features_mean[0].shape[1])
print("Image-Text features weighted dimension:", features_weighted[0].shape[1])

Text features dimension: 512
Image features dimension: 512
Image-Text features mean dimension: 512
Image-Text features weighted dimension: 512


In [ ]:
base_data

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0...."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ..."
...,...,...,...,...,...,...,...,...,...
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879,"[[0.4673, 0.1345, 0.04214, -0.2769, 0.1053, -0...","[[-0.04492, 0.182, 0.3486, -0.03268, -0.05658,...","[[0.2112, 0.1582, 0.1954, -0.1548, 0.02435, 0....","[[0.2625, 0.1536, 0.1647, -0.1792, 0.04053, 0...."
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0...."
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0..."
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ...","[[0.0874, -0.1564, 0.09424, 0.1768, 0.284, 0.0...","[[0.10846, -0.1841, 0.0896, 0.22, 0.283, 0.012..."


## Transform the Music Format

#### .WAV File -> .MID File

In [ ]:
import os
import librosa
import crepe
import pretty_midi

def convert_wav_to_midi(wav_path, midi_path):
    try:
        # 加载音频文件
        audio, sr = librosa.load(wav_path)

        # 使用 CREPE 进行音高预测
        time, frequency, confidence, _ = crepe.predict(audio, sr, viterbi=True)

        # 创建 PrettyMIDI 对象
        midi = pretty_midi.PrettyMIDI()
        instrument = pretty_midi.Instrument(program=0)

        # 遍历预测的时间、频率和置信度
        for t, f, c in zip(time, frequency, confidence):
            if c > 0.8:  # 置信度阈值
                note_number = librosa.hz_to_midi(f)  # 将频率转换为 MIDI 音符编号
                start = t
                end = t + 0.05  # 短时持续时间
                note = pretty_midi.Note(velocity=100, pitch=int(note_number), start=start, end=end)
                instrument.notes.append(note)

        # 将音符添加到 MIDI 对象中并保存
        midi.instruments.append(instrument)
        midi.write(midi_path)
        print(f"Converted {wav_path} to {midi_path}")
    except Exception as e:
        print(f"Failed to convert {wav_path}: {e}")

def batch_convert(wav_folder, midi_folder):
    if not os.path.exists(midi_folder):
        os.makedirs(midi_folder)

    for wav_file in os.listdir(wav_folder):
        if wav_file.endswith('.wav'):
            wav_path = os.path.join(wav_folder, wav_file)
            midi_path = os.path.join(midi_folder, wav_file.replace('.wav', '.mid'))
            convert_wav_to_midi(wav_path, midi_path)


batch_convert(wav_folder, midi_folder)


32/32 [==============================] - 1s 16ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-0SdAVK79lg_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-0SdAVK79lg_30-40.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-0vPFx-wRRI_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-0vPFx-wRRI_30-40.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-0xzrMun0Rs_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-0xzrMun0Rs_30-40.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-3Kv4fdm7Uk_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-3Kv4fdm7Uk_30-40

E:\conda_pkg\.conda\envs\newEnv\lib\site-packages\crepe\core.py:209: RuntimeWarning: invalid value encountered in true_divide
  frames /= np.std(frames, axis=1)[:, np.newaxis]


32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-ByoSbgzr4M_0-10.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-ByoSbgzr4M_0-10.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-cLzki-B06o_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-cLzki-B06o_30-40.mid
32/32 [==============================] - 0s 8ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-Dtir74TiUM_40-50.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-Dtir74TiUM_40-50.mid
20/32 [=================>............] - ETA: 0s

E:\conda_pkg\.conda\envs\newEnv\lib\site-packages\crepe\core.py:209: RuntimeWarning: invalid value encountered in true_divide
  frames /= np.std(frames, axis=1)[:, np.newaxis]


32/32 [==============================] - 0s 15ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-e4wXAy1iVo_0-10.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-e4wXAy1iVo_0-10.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-eDAoheZrY8_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-eDAoheZrY8_30-40.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-EVRXQpt1-8_50-60.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-EVRXQpt1-8_50-60.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\-f1DNyngKVY_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\-f1DNyngKVY_30-40.m

...

32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\z_fEIZOv0JY_0-10.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\z_fEIZOv0JY_0-10.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\Z_HT-d8W1_M_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\Z_HT-d8W1_M_30-40.mid
32/32 [==============================] - 0s 8ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_-kssA-FOzU_300-310.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_-kssA-FOzU_300-310.mid
32/32 [==============================] - 0s 8ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_0-2meOf9qY_10-20.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_0-2meOf9qY_10-2

E:\conda_pkg\.conda\envs\newEnv\lib\site-packages\crepe\core.py:209: RuntimeWarning: invalid value encountered in true_divide
  frames /= np.std(frames, axis=1)[:, np.newaxis]


32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_IkLUOsNHAA_0-10.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_IkLUOsNHAA_0-10.mid
32/32 [==============================] - 0s 8ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_IP6zlayY7k_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_IP6zlayY7k_30-40.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_JTHtcRHxnw_300-310.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_JTHtcRHxnw_300-310.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_KaRkSyELy4_490-500.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_KaRkSyELy4_49

E:\conda_pkg\.conda\envs\newEnv\lib\site-packages\crepe\core.py:209: RuntimeWarning: invalid value encountered in true_divide
  frames /= np.std(frames, axis=1)[:, np.newaxis]


32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_Ra1Y6K7nSs_0-10.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_Ra1Y6K7nSs_0-10.mid
32/32 [==============================] - 0s 8ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_RLsXrr0fQo_80-90.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_RLsXrr0fQo_80-90.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_Rpn2FUAUgY_20-30.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_Rpn2FUAUgY_20-30.mid
32/32 [==============================] - 0s 9ms/step
Converted E:\Project and Dissertation in Data Science\dataset\archive\musiccaps\_RrA-0lfIiU_30-40.wav to E:\Project and Dissertation in Data Science\dataset\archive\music_midi\_RrA-0lfIiU_30-40.mi

### Load Melodies from MIDI Files

In [ ]:
# 从数据框中的文件名找到对应的MIDI文件并进行转换
def load_melodies_from_midi_files(df, midi_folder_path):
    melodies = []
    for filename in df['Music_Name']:
        midi_file_path = os.path.join(midi_folder_path, f"{filename}.mid")
        if os.path.exists(midi_file_path):
            melody = note_seq.midi_file_to_melody(midi_file_path)
            melodies.append(melody)
        else:
            print(f"File {midi_file_path} does not exist.")
    return melodies

# 加载MIDI文件并转换为Melody
melodies = load_melodies_from_midi_files(test_data, midi_folder)

### All Data for Melodies

In [ ]:
# 从数据框中的文件名找到对应的MIDI文件并进行转换
def load_melodies_from_midi_files(df, midi_folder_path):
    melodies = []
    for filename in df['Music_Name']:
        midi_file_path = os.path.join(midi_folder_path, f"{filename}.mid")
        if os.path.exists(midi_file_path):
            melody = note_seq.midi_file_to_melody(midi_file_path)
            melodies.append(melody)
        else:
            print(f"File {midi_file_path} does not exist.")
    return melodies

# 加载MIDI文件并转换为Melody
melodies = load_melodies_from_midi_files(base_data, midi_folder)

In [ ]:
base_data2 = base_data
base_data2['melody'] = melodies

# 序列化 Melody 对象为 JSON 字符串
base_data2['melody_json'] = base_data2['melody'].apply(lambda x: json.dumps(x._events))
# 创建 HDF5 文件（'w' 代表写模式）
with h5py.File('data_with_melody.h5', 'w') as hf:
    hf.create_dataset('image_features', data=np.array(base_data2['image_features'].tolist()))
    hf.create_dataset('text_features', data=np.array(base_data2['text_features'].tolist()))
    hf.create_dataset('features_mean', data=np.array(base_data2['features_mean'].tolist()))
    hf.create_dataset('features_weighted', data=np.array(base_data2['features_weighted'].tolist()))
    hf.create_dataset('melody', data=np.array(base_data2['melody_json'].tolist()).astype('S'))

### Load Whole Dataset Melodies

In [ ]:
# 读取 HDF5 文件
with h5py.File('data_with_melody.h5', 'r') as hf:
    melody_data = hf['melody'][:]
    # 反序列化 JSON 字符串为 Melody 对象
    melodies = [note_seq.Melody(json.loads(m.decode())) for m in melody_data]

    image_features = hf['image_features'][:]
    text_features = hf['text_features'][:]
    features_mean = hf['features_mean'][:]
    features_weighted = hf['features_weighted'][:]

# 将数组转换为列表（如果需要）并添加到 other_data DataFrame
base_data['image_features'] = list(image_features)
base_data['text_features'] = list(text_features)
base_data['features_mean'] = list(features_mean)
base_data['features_weighted'] = list(features_weighted)
base_data['melody'] = list(melodies)

In [ ]:
base_data

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted,melody
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542...","(50, -1, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0...","(-2, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378...","(-2, -2, -2, -2, -2, -2, 57, 56, 56, 55, 55, -..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0....","(67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, -..."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ...","(-2, -2, 60, 60, -1, -2, -2, -2, 60, 60, 60, -..."
...,...,...,...,...,...,...,...,...,...,...
443657,willi-baumeister_machine-man-with-spiral-turn-...,The interlocking mechanical shapes fitting tog...,R_HAtyDbw1M_30-40,Someone is skillfully playing maracas playing ...,0.806879,"[[0.4673, 0.1345, 0.04214, -0.2769, 0.1053, -0...","[[-0.04492, 0.182, 0.3486, -0.03268, -0.05658,...","[[0.2112, 0.1582, 0.1954, -0.1548, 0.02435, 0....","[[0.2625, 0.1536, 0.1647, -0.1792, 0.04053, 0....",()
443658,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0....","(-2, -2, 69, 69, -1, 64, -1, -2, -2, 71, 71, 7..."
443659,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0...","(63, 63, 60, 60, 63, 63, 63, 65, 65, 63, 62, 6..."
443660,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ...","[[0.0874, -0.1564, 0.09424, 0.1768, 0.284, 0.0...","[[0.10846, -0.1841, 0.0896, 0.22, 0.283, 0.012..

In [ ]:
count = 0
for melody in melodies:
    # 检查每个 Melody 对象的 _events 属性是否为空列表
    if not melody._events:  # 如果列表为空，这个表达式为真
        count += 1

print('空的 Melody 对象数量: ', count)

空的 Melody 对象数量:  20901


In [ ]:
base_data.index

RangeIndex(start=0, stop=443662, step=1)

In [ ]:
base_data['is_melody_empty'] = base_data['melody'].apply(lambda x: len(x._events) == 0)
base_data = base_data[base_data['is_melody_empty'] == False]
base_data.drop(columns=['is_melody_empty'], inplace=True)

C:\Users\hjy\AppData\Local\Temp\ipykernel_6904\2148986185.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_data.drop(columns=['is_melody_empty'], inplace=True)


In [ ]:
base_data = base_data.reset_index(drop=True)
base_data.index

RangeIndex(start=0, stop=422761, step=1)

In [ ]:
processed_base_data = base_data.iloc[:,:5]

In [ ]:
processed_base_data.to_csv("processed_music_matched_data.csv", index=False)

In [ ]:
# Load processed matched musical data, which has deleted the no-sound audio row of data
base_data2 = pd.read_csv("processed_music_matched_data.csv")
base_data2

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201
...,...,...,...,...,...
422756,david-burliuk_landscape-1,The greenery landscape and flowery background ...,M0ygCD6WyXw_0-10,This clip consists of a blowing horn being pla...,0.758374
422757,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300
422758,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008
422759,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153


In [ ]:
# 序列化 Melody 对象为 JSON 字符串
base_data['melody_json'] = base_data['melody'].apply(lambda x: json.dumps(x._events))
# 创建 HDF5 文件（'w' 代表写模式）
with h5py.File('processed_data_with_melody.h5', 'w') as hf:
    hf.create_dataset('image_features', data=np.array(base_data['image_features'].tolist()))
    hf.create_dataset('text_features', data=np.array(base_data['text_features'].tolist()))
    hf.create_dataset('features_mean', data=np.array(base_data['features_mean'].tolist()))
    hf.create_dataset('features_weighted', data=np.array(base_data['features_weighted'].tolist()))
    hf.create_dataset('melody', data=np.array(base_data['melody_json'].tolist()).astype('S'))

In [ ]:
base_data

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted,melody,melody_json
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542...","(50, -1, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5...","[50, -1, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0...","(-2, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3...","[-2, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378...","(-2, -2, -2, -2, -2, -2, 57, 56, 56, 55, 55, -...","[-2, -2, -2, -2, -2, -2, 57, 56, 56, 55, 55, -..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0....","(67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, -...","[67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, -..."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ...","(-2, -2, 60, 60, -1, -2, -2, -2, 60, 60, 60, -...","[-2, -2, 60, 60, -1, -2, -2, -2, 60, 60, 60, -..."
...,...,...,...,...,...,...,...,...,...,...,...
422756,david-burliuk_landscape-1,The greenery landscape and flowery background ...,M0ygCD6WyXw_0-10,This clip consists of a blowing horn being pla...,0.758374,"[[0.2013, -0.2996, 0.1554, -0.04733, 0.1796, -...","[[0.06445, 0.1355, 0.3389, -0.1179, -0.04034, ...","[[0.1328, -0.08203, 0.2471, -0.08264, 0.0696, ...","[[0.1466, -0.1256, 0.2288, -0.07556, 0.0916, -...","(-2, 66, 66, 59, 59, 59, 59, 59, 59, 59, 59, 5...","[-2, 66, 66, 59, 59, 59, 59, 59, 59, 59, 59, 5..."
422757,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0....","(-2, -2, 69, 69, -1, 64, -1, -2, -2, 71, 71, 7...","[-2, -2, 69, 69, -1, 64, -1, -2, -2, 71, 71, 7..."
422758,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0...","(63, 63, 60, 60, 63,

In [ ]:
# 读取 HDF5 文件
with h5py.File('processed_data_with_melody.h5', 'r') as hf:
    melody_data = hf['melody'][:]
    # 反序列化 JSON 字符串为 Melody 对象
    melodies = [note_seq.Melody(json.loads(m.decode())) for m in melody_data]

    image_features = hf['image_features'][:]
    text_features = hf['text_features'][:]
    features_mean = hf['features_mean'][:]
    features_weighted = hf['features_weighted'][:]

# 将数组转换为列表（如果需要）并添加到 other_data DataFrame
base_data2['image_features'] = list(image_features)
base_data2['text_features'] = list(text_features)
base_data2['features_mean'] = list(features_mean)
base_data2['features_weighted'] = list(features_weighted)
base_data2['melody'] = list(melodies)

In [ ]:
base_data2

,Artwork,Art_Utterance,Music_Name,Music_Comment,Similarity_Score,image_features,text_features,features_mean,features_weighted,melody
0,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"She seems very happy in the picture, and you w...",ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.791458,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0315, 0.1757, -0.1968, 0.0465, -0.04526, ...","[[0.1167, 0.002075, -0.0323, 0.03006, 0.2042, ...","[[0.1464, -0.03265, 0.0006714, 0.02676, 0.2542...","(50, -1, 50, 50, 50, 50, 50, 50, 50, 50, 50, 5..."
1,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,This woman has really knotty hands which makes...,vnwKpQeza3A_320-330,This is a recording of two didgeridoos. They a...,0.772168,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.0724, 0.3037, -0.4678, -0.1588, 0.1721, 0...","[[0.09625, 0.0661, -0.1677, -0.07263, 0.313, 0...","[[0.13, 0.01855, -0.10767, -0.0554, 0.341, 0.0...","(-2, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 3..."
2,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"When looking at this woman, I am filled with c...",0VwX92X3iPc_30-40,This audio contains a female voice speaking in...,0.798202,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.05783, -0.0409, -0.4458, -0.0473, -0.08594...","[[0.1614, -0.1062, -0.1567, -0.01685, 0.1838, ...","[[0.1821, -0.11926, -0.0989, -0.010765, 0.2378...","(-2, -2, -2, -2, -2, -2, 57, 56, 56, 55, 55, -..."
3,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,"A woman looking at ease, peaceful, and satisfi...",kh6rmFg3U4k_480-490,The low quality recording features a resonatin...,0.792188,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[0.1517, -0.2998, 0.1375, 0.3303, 0.3237, -0....","[[0.2083, -0.2356, 0.1348, 0.172, 0.3887, -0.3...","[[0.2196, -0.2228, 0.1343, 0.1403, 0.4019, -0....","(67, 67, 67, 67, 67, 67, 67, 67, 67, 66, 66, -..."
4,vincent-van-gogh_portrait-of-madame-ginoux-l-a...,She looks like a lady from that past that migh...,-VI2IRq17rs_360-370,"In this clip, a large bell is rung and left to...",0.740201,"[[0.265, -0.1715, 0.1322, 0.013596, 0.4536, -0...","[[-0.1611, -0.002035, -0.2603, 0.1305, 0.1753,...","[[0.05188, -0.0868, -0.064, 0.072, 0.3145, -0....","[[0.0945, -0.1037, -0.02472, 0.06033, 0.3423, ...","(-2, -2, 60, 60, -1, -2, -2, -2, 60, 60, 60, -..."
...,...,...,...,...,...,...,...,...,...,...
422756,david-burliuk_landscape-1,The greenery landscape and flowery background ...,M0ygCD6WyXw_0-10,This clip consists of a blowing horn being pla...,0.758374,"[[0.2013, -0.2996, 0.1554, -0.04733, 0.1796, -...","[[0.06445, 0.1355, 0.3389, -0.1179, -0.04034, ...","[[0.1328, -0.08203, 0.2471, -0.08264, 0.0696, ...","[[0.1466, -0.1256, 0.2288, -0.07556, 0.0916, -...","(-2, 66, 66, 59, 59, 59, 59, 59, 59, 59, 59, 5..."
422757,gino-severini_a-dancer-1,the collection and collage of different colors...,oMZcsGUi8ZE_0-10,This clip features a synchronised playing of s...,0.799300,"[[0.282, 0.1711, 0.1952, -0.1887, 0.417, 0.089...","[[-0.05054, 0.3774, 0.2979, -0.06555, -0.04782...","[[0.1157, 0.2744, 0.2466, -0.1272, 0.1846, -0....","[[0.1489, 0.2537, 0.2363, -0.1395, 0.2311, -0....","(-2, -2, 69, 69, -1, 64, -1, -2, -2, 71, 71, 7..."
422758,ivan-aivazovsky_sea-at-night-1861,The peaceful reflections of the moonlight on t...,s1QeDT7jqHQ_30-40,The low quality recording features multiple la...,0.781008,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[0.0625, -0.1382, 0.1168, 0.2683, 0.3545, -0....","[[0.1274, -0.2164, 0.0941, 0.33, 0.317, -0.103...","[[0.1405, -0.2322, 0.08954, 0.3428, 0.3093, -0...","(63, 63, 60, 60, 63, 63, 63, 65, 65, 63, 62, 6..."
422759,ivan-aivazovsky_sea-at-night-1861,I can imagine the sailors resting this peacefu...,ABVYSaLu_VM_10-20,Here we have a slow piano piece played in a ma...,0.733153,"[[0.1924, -0.2947, 0.07135, 0.392, 0.2793, -0....","[[-0.01753, -0.018, 0.11707, -0.0385, 0.2886, ...","[[0.0874, -0.1564, 0.09424, 0.1768, 0.284, 0.0...","[[0.10846, -0.1841, 0.08